<a href="https://colab.research.google.com/github/ajohn256/Deep-Learning-With-Pytorch/blob/main/CNN_BRCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import torch.nn.functional as F
import shutil

In [120]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [8]:
# Define data transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Grayscale(num_output_channels=1),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        # transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [9]:
# Define the data directory
data_dir = './BRCA_DATASET'

# # Create data loaders
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}


In [97]:
# Remove .ipynb_checkpoints directory if it exists
for split in ['train', 'test']:
    checkpoint_dir = os.path.join(data_dir, split, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)  # Remove the directory

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [98]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 16
     Root location: ./BRCA_DATASET/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 16
     Root location: ./BRCA_DATASET/test
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [99]:
#image_datasets
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

{'train': 16, 'test': 16}


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


['benign', 'malignant']

In [100]:
train_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=30, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=30, shuffle=True, num_workers=2)

In [101]:
image,label = image_datasets['train'][0]
image.size()

torch.Size([3, 224, 224])

In [102]:
# conv1 = nn.Conv2d(3,6,3,1)
# conv2 = nn.Conv2d(6,16,3,1)

In [103]:
# for i, (X_Train,y_train) in enumerate(image_datasets['train']):
#     break

In [104]:
# X_Train.shape#get shape of each image

In [105]:
# x = X_Train.view(1,3,224,224)

In [106]:
# x.shape

In [107]:
# x = F.relu(conv1(x))

In [108]:
#pooling
# x = F.max_pool2d(x,2,2)
#

In [109]:
# x.shape

In [110]:
# x = F.relu(conv2(x))

In [111]:
# x.shape

In [112]:
# x = F.max_pool2d(x,2,2)
# x.shape

In [113]:
class_names = ['benign', 'malignant']

In [118]:
class NeuralBRCA(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.pool = nn.MaxPool2d(2,2)

       #fully connected layer
        self.fc1 = nn.Linear(54*54*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x,1)
        x = x.view(-1,54*54*16)#flatten

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [121]:
net = NeuralBRCA()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
model = net.to(device)

In [122]:
def train_one():
  for epoch in range(30):
      print(f"Training epoch {epoch}...")
      running_loss = 0.0

      for i, data in enumerate(train_loader):
          inputs,labels = data
          optimizer.zero_grad()
          outputs = net(inputs)

          loss = loss_function(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Loss: {running_loss / len(train_loader):.4f}")


In [127]:
def train_two():
  # Training loop
  num_epochs = 30
  for epoch in range(num_epochs):
      for phase in ['train', 'test']:
          if phase == 'train':
              model.train()
          else:
              model.eval()

          running_loss = 0.0
          running_corrects = 0

          for inputs, labels in dataloaders[phase]:
              inputs = inputs.to(device)
              labels = labels.to(device)

              optimizer.zero_grad()

              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = loss_function(outputs, labels)

                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

  print("Training complete!")

In [128]:
train_two()

train Loss: 2.1114 Acc: 0.5000
test Loss: 2.0647 Acc: 0.8125
train Loss: 2.0802 Acc: 0.6250
test Loss: 2.0224 Acc: 0.8125
train Loss: 2.0335 Acc: 0.6875
test Loss: 1.9715 Acc: 0.8125
train Loss: 1.9889 Acc: 0.8750
test Loss: 1.9099 Acc: 0.8125
train Loss: 1.9334 Acc: 0.7500
test Loss: 1.8349 Acc: 0.8125
train Loss: 1.8591 Acc: 0.8125
test Loss: 1.7423 Acc: 0.8125
train Loss: 1.7607 Acc: 0.8750
test Loss: 1.6284 Acc: 0.8125
train Loss: 1.6564 Acc: 0.7500
test Loss: 1.4897 Acc: 0.8125
train Loss: 1.5222 Acc: 0.8125
test Loss: 1.3259 Acc: 0.8125
train Loss: 1.3426 Acc: 0.7500
test Loss: 1.1405 Acc: 0.8125
train Loss: 1.1692 Acc: 0.8125
test Loss: 0.9467 Acc: 0.8125
train Loss: 0.9643 Acc: 0.8125
test Loss: 0.7658 Acc: 0.8125
train Loss: 0.8002 Acc: 0.8125
test Loss: 0.6172 Acc: 0.8125
train Loss: 0.6354 Acc: 0.8125
test Loss: 0.5016 Acc: 0.9375
train Loss: 0.5106 Acc: 1.0000
test Loss: 0.4157 Acc: 0.9375
train Loss: 0.4275 Acc: 1.0000
test Loss: 0.3608 Acc: 1.0000
train Loss: 0.3502 Acc: 

In [129]:
torch.save(net.state_dict(),'./trained_net.pth')

In [130]:
net = NeuralBRCA()
net.load_state_dict(torch.load('./trained_net.pth'))

<ipython-input-130-aaf33257af9f>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('./trained_net.pth'))


<All keys matched successfully>

In [139]:
new_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Grayscale(num_output_channels=1),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [140]:
from PIL import Image
def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

In [147]:
img1 = "./ml1.jpg"
img2 = "./mal2.jpg"
img3 = "./B1.png"
img4 = "./B2.png"
images = [img1,img2,img3]

In [148]:
# net.eval()
# with torch.no_grad():
#   for image in images:
#     output = net(image)
#     _,predicted = torch.max(output,1)
#     print(f"Predicted class: {class_names[predicted.item()]}")

In [149]:
net.eval()
with torch.no_grad():
  for image_path in images:  # Iterate through image paths
    image_tensor = load_image(image_path)  # Load image and transform to Tensor
    output = net(image_tensor)  # Pass the Tensor to the model
    _,predicted = torch.max(output,1)
    print(f"Predicted class: {class_names[predicted.item()]}")

Predicted class: malignant
Predicted class: benign
Predicted class: benign
